In [2]:
from google.colab import drive
import shutil
drive.mount('/content/drive')
shutil.copyfile("/content/drive/MyDrive/RetailRecommender/preprocessing.py", "preprocessing.py")
shutil.copyfile("/content/drive/MyDrive/RetailRecommender/Datasets/online_retail_processed.csv", "sample_data/online_retail_processed.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'sample_data/online_retail_processed.csv'

In [3]:
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch-geometric
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 58.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 KB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 18.9 MB/s eta 0:00:00
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=ee48958044178d0e2533b10b03197ce47ccb96386251d3fa7ac20619feda44cc
  Stored in directory: /root/.cache/pip/wheels/59/a3/20/198928106d3169865ae73afcbd3d3d1796cf6b429b55c65378
Successfully built 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=aad1a12c735a80ef69086cd7c59e41d31c36d5519b89b41980f9591b24a73890
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [4]:
# Initialization
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from torch.utils.data import Dataset, DataLoader
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from sentence_transformers import SentenceTransformer

from preprocessing import split_temporal
device = "cuda:0" if torch.cuda.is_available() else "cpu"

The Dataset

In [5]:
df = pd.read_csv('sample_data/online_retail_processed.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,CountryID,StockCodeID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 8:26,2.55,0.0,United Kingdom,0,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 8:26,3.39,0.0,United Kingdom,0,1
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 8:26,2.75,0.0,United Kingdom,0,2
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 8:26,3.39,0.0,United Kingdom,0,3
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 8:26,3.39,0.0,United Kingdom,0,4


In [ ]:
df.describe()

,Quantity,UnitPrice,CustomerID,CountryID,StockCodeID
count,537363.000000,537363.000000,537363.000000,537363.000000,537363.000000
mean,9.854074,4.224191,1148.064805,0.617235,1335.866269
std,216.377933,72.455634,1230.025049,2.850985,1196.756401
min,-80995.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.250000,69.000000,0.000000,310.000000
50%,3.000000,2.080000,673.000000,0.000000,912.000000
75%,10.000000,4.130000,1929.000000,0.000000,2166.000000
max,80995.000000,17836.460000,4362.000000,37.000000,3904.000000


In [ ]:
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = sbert_model.encode(df['Description'])

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

KeyboardInterrupt: ignored

In [6]:
train_df, test_df = split_temporal(df, "InvoiceDate")

In [ ]:
train_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,CountryID,StockCodeID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,0.0,United Kingdom,0,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,0.0,United Kingdom,0,1
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,0.0,United Kingdom,0,2
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,0.0,United Kingdom,0,3
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,0.0,United Kingdom,0,4
...,...,...,...,...,...,...,...,...,...,...
429865,573937,22029,SPACEBOY BIRTHDAY CARD,12,2011-11-02 10:40:00,0.42,4014.0,United Kingdom,0,1347
429866,573937,22045,SPACEBOY GIFT WRAP,25,2011-11-02 10:40:00,0.42,4014.0,United Kingdom,0,1461
429867,573937,23232,WRAP VINTAGE LEAF DESIGN,25,2011-11-02 10:40:00,0.42,4014.0,United Kingdom,0,2897
429868,573937,16169E,WRAP 50'S CHRISTMAS,25,2011-11-02 10:40:00,0.42,4014.0,United Kingdom,0,3557


In [7]:
class RetailData(Dataset):
  def __init__(self, df):
    x = df.iloc[:,:-1]
    y = df.iloc[:, -1]
    self.x_train=torch.tensor(x.values,dtype=torch.float32)
    self.y_train=torch.tensor(y.values,dtype=torch.float32)
    
  def __len__(self):
    return len(self.y_train)

  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]

In [8]:
print(df.shape, train_df.shape, test_df.shape)
print(df['CustomerID'].unique().shape, train_df['CustomerID'].unique().shape, test_df['CustomerID'].unique().shape)
print(df['CustomerID'].max(), train_df['CustomerID'].max(), test_df['CustomerID'].max())

(537363, 10) (429890, 10) (107473, 10)
(4363,) (4015,) (1920,)
4362.0 4014.0 4362.0


In [9]:
train_loader = RetailData(train_df[["Quantity", "UnitPrice", "CustomerID", "CountryID", "StockCodeID"]])

In [11]:
total_len = len(df['CustomerID'].unique()) + len(df['StockCodeID'].unique())
num_users = len(df['CustomerID'].unique())
tr_sentence_embeddings = sbert_model.encode(train_df['Description'])

x = torch.cat((torch.tensor(range(0, total_len)).unsqueeze(1), tr_sentence_embeddings), dim=1)
edge_index = torch.tensor([train_df['CustomerID'], train_df['StockCodeID'] + num_users]).long()
edge_attr = torch.tensor([train_df['UnitPrice'] * train_df['Quantity']]).T
avg_edge_weight = edge_attr.mean()
print(x.shape, edge_index.shape, edge_attr.shape, avg_edge_weight)
tr_data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

torch.Size([8268, 129]) torch.Size([2, 429890]) torch.Size([429890, 1]) tensor(18.5253, dtype=torch.float64)


In [12]:
edge_index.max()

tensor(8215)

In [14]:
te_sentence_embeddings = sbert_model.encode(test_df['Description'])

te_x = torch.cat((torch.tensor(range(0, total_len)).unsqueeze(1), te_sentence_embeddings), dim=1)
te_edge_index = torch.tensor([(test_df["CustomerID"]).values, (test_df["StockCodeID"] + num_users).values]).long()
te_edge_attr = torch.tensor([test_df['UnitPrice'].values * test_df['Quantity'].values]).T
te_data = Data(x=te_x, edge_index=te_edge_index, edge_attr=te_edge_attr)

<ipython-input-14-0f38c3a3967c>:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  te_edge_index = torch.tensor([(test_df["CustomerID"]).values, (test_df["StockCodeID"] + num_users).values]).long()


In [15]:
print(edge_index.shape)
print(te_edge_index.shape)

torch.Size([2, 429890])
torch.Size([2, 107473])


Customer and Item Nodes

In [16]:
item_ids = torch.tensor(df['StockCodeID'].unique())
customer_ids = torch.tensor(df['CustomerID'].unique())
print(item_ids.shape, customer_ids.shape)

torch.Size([3905]) torch.Size([4363])


In [17]:
from torch_geometric.utils.negative_sampling import negative_sampling
import torch_geometric as ptg
import torch.nn.functional as F
from torch_geometric.utils import degree

In [18]:
class GraphEmbedder(nn.Module):
  def __init__(self, embed_size, num_layers, num_nodes, dropout_p=0.5):
    super().__init__()
    self.embed = nn.Embedding(num_nodes, embed_size)
    self.projection = nn.Linear(128, embed_size)
    layers = [ptg.nn.GCNConv(embed_size, embed_size) for _ in range(num_layers)]
    self.layers = nn.ModuleList(layers)
    self.dropout_p = dropout_p
  
  def forward(self, data):
    x,edge_indices = data.x, data.edge_index
    bert = x[:, 1:]
    item_embeds = self.projection(bert)
    x = self.embed(x[:,0].long())
    x = torch.cat((x, item_embeds), dim=0)
    for i, layer in enumerate(self.layers):
      dropped = F.dropout(x, p=self.dropout_p, training=self.training)
      x = layer(dropped, edge_indices)
      if i != len(self.layers) - 1:
        x = F.relu(x)
    return x

In [19]:
def recall_k(net, data, k):
  net.eval()
  embeds = net(data)
  user_embeds, item_embeds = embeds[:num_users], embeds[num_users:]
  dots = F.logsigmoid(user_embeds @ item_embeds.T)
  top_k = torch.topk(dots, k, dim=1).indices
  neighs = degree(data.edge_index[0])
  total = 0
  for u in range(num_users):
    overlap_amt = len(np.intersect1d(
        top_k[u].detach().cpu().numpy(),
        torch.where(data.edge_index[0] == u, 
        data.edge_index[1],
        -1).detach().cpu().numpy())
    )
    total += overlap_amt/neighs[u]
  return total/num_users

In [20]:
def train(encoder, optim, epochs, tr_data, te_data):
    tr_losses = []
    te_losses = []
    te_recalls = []
    best_recall = -np.inf
    bad_epochs = 0
    for i in range(epochs):
      embedded = encoder(tr_data)
      user_embeds, item_embeds = embedded[:num_users], embedded[num_users:]
      pos = tr_data.edge_index
      neg_users = torch.randint(0, num_users, (len(tr_data.edge_index),))
      neg_items = torch.randint(num_users, total_len, (len(tr_data.edge_index),))
      dots = user_embeds @ item_embeds.T
      pos_weights = F.logsigmoid(dots[pos[0], pos[1]-num_users]*tr_data.edge_attr[0])
      neg_weights = F.logsigmoid(dots[neg_users, neg_items-num_users]*avg_edge_weight)

      loss = pos_weights.sum() - neg_weights.sum()
      loss.backward()
      optim.step()
      optim.zero_grad()

      # epoch testing
      embedded = encoder(te_data)
      user_embeds, item_embeds = embedded[:num_users], embedded[num_users:]
      pos = te_data.edge_index
      neg_users = torch.randint(0, num_users, (len(te_data.edge_index),))
      neg_items = torch.randint(num_users, total_len, (len(te_data.edge_index),))
      dots = user_embeds @ item_embeds.T
      pos_weights = F.logsigmoid(dots[pos[0], pos[1]-num_users]*te_data.edge_attr[0])
      neg_weights = F.logsigmoid(dots[neg_users, neg_items-num_users]*avg_edge_weight)

      te_loss = pos_weights.sum() - neg_weights.sum()
      
      tr_losses.append(loss.item())
      te_recalls.append(te_loss.item())
      te_recalls.append(recall_k(encoder, te_data, 150).item())
      if i % 10 == 9:
          print(tr_losses[-1])
    fig, (ax1, ax2, ax3) = plt.subplots(2)
    fig.set_figheight(15)
    fig.set_figwidth(15)
    ax1.plot(tr_losses, label="Train Loss", color="#cc6462")
    ax1.plot(te_losses, label="Valid Loss", color="#9e9e9e") 
    ax1.set(xlabel="Iterations")
    ax1.set_title("Training and Validation Losses")
    ax1.legend()

    ax3.plot(te_recalls, color="#cc6462")
    ax3.set_title("Recall 150")

In [21]:
embed_net = GraphEmbedder(32, 2, total_len)
optimizer = torch.optim.Adam(embed_net.parameters())

In [ ]:
train(embed_net, optimizer, 200, tr_data, te_data)

-218618.96863038535
-7125442.871003504


In [ ]:
embed_net.eval()
test_user = np.random.randint(num_users)
embeds = embed_net(tr_data)
user_embeds, item_embeds = embeds[:num_users], embeds[num_users:]
dots = F.logsigmoid(user_embeds @ item_embeds.T)
top_k = torch.topk(dots, 10, dim=1).indices
recommends = top_k[test_user]
print(recommends)

In [ ]:
for idx in recommends:
  print(list(df.loc[df["StockCodeID"] == idx.item()]["Description"])[0])